In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
import sys
sys.path.append('../src')

from memory_pair import MemoryPair

In [3]:
import numpy as np

# ----------------------------------------
# helper: Sherman–Morrison utilities
def sm_add_inv(H_inv, u):
    """
    Rank-1 *addition* :   H_new = H + u uᵀ
    Returns updated inverse H_inv_new.
    """
    Hu = H_inv @ u
    denom = 1.0 + u.T @ Hu
    return H_inv - np.outer(Hu, Hu) / denom

def sm_remove_inv(H_inv, u):
    """
    Rank-1 *downdate* :   H_new = H − u uᵀ
    (caller must ensure denominator > 0)
    """
    Hu = H_inv @ u
    denom = 1.0 - u.T @ Hu           # must stay positive
    return H_inv + np.outer(Hu, Hu) / denom
# ----------------------------------------


class StreamNewtonMemoryPair:
    """
    Online ridge-regression learner + unlearner with
    (ε,δ)-style Gaussian noise per deletion.

    Loss      : ½(θᵀx − y)²   (squared error)
    Regulariser: λ‖θ‖₂²
    """

    def __init__(self, dim, lam=1.0,
                 eps_total=1.0,   delta_total=1e-5,
                 max_deletions=20):
        self.dim  = dim
        self.lam  = lam

        # Model parameters & (regularised) Hessian inverse
        self.theta = np.zeros(dim)
        self.H_inv = np.eye(dim) / lam          # (XᵀX + λI)⁻¹

        # Storage: keep raw (x,y) for exact gradient recomputation
        self.data_store = {}     # id -> (x, y)
        self.deleted     = set()

        # --- privacy bookkeeping ---
        self.K          = max_deletions          # anticipate ≤K deletions
        self.eps_total  = eps_total
        self.delta_total = delta_total
        self.eps_step   = eps_total  / (2*max_deletions)   # split budget
        self.delta_step = delta_total / (2*max_deletions)
        self.eps_spent  = 0.0

    # 1st-order gradient (current θ)
    def grad(self, x, y):
        err = self.theta @ x - y
        return err * x

    # ---------------- insert ----------------
    def insert(self, idx, x, y):
        """Process a new data point (x,y)."""
        if idx in self.deleted or idx in self.data_store:
            raise ValueError("duplicate id")

        # Update inverse Hessian   H⁻¹ ← (H + x xᵀ)⁻¹  via Sherman–Morrison
        self.H_inv = sm_add_inv(self.H_inv, x)

        # One Newton step using fresh gradient
        g = self.grad(x, y)
        self.theta -= self.H_inv @ g

        # Store raw data for possible future deletion
        self.data_store[idx] = (x, y)

    # ---------------- delete ----------------
    def delete(self, idx):
        """Remove the influence of data point idx (if present)."""
        if idx in self.deleted or idx not in self.data_store:
            return

        x, y = self.data_store.pop(idx)

        # ---------------- Hessian downdate ----------------
        self.H_inv = sm_remove_inv(self.H_inv, x)

        # Re-compute gradient of this point *at current θ*
        g = self.grad(x, y)

        delta_theta = self.H_inv @ g       # influence to remove
        self.theta -= delta_theta

        # ---------------- calibrated Gaussian noise ----------------
        sensitivity = np.linalg.norm(delta_theta, 2)
        sigma = (sensitivity *
                 np.sqrt(2*np.log(1.25/self.delta_step))
                 / self.eps_step)

        self.theta += np.random.normal(0.0, sigma, size=self.dim)

        # Budget accounting
        self.eps_spent += self.eps_step
        self.deleted.add(idx)

    # ---------------- utility ----------------
    def privacy_ok(self):
        return self.eps_spent <= self.eps_total

In [4]:
import numpy as np
import scipy.stats as st
from memory_pair import MemoryPair
import os

def generate_synthetic_data(n_samples=1000, n_features=10, noise=0.5):
    """Generates synthetic data for a linear regression problem."""
    X = np.random.rand(
        n_samples, 
        n_features
    )
    true_w = np.random.randn(n_features)
    y = X @ true_w + np.random.normal(
        0, 
        noise, 
        n_samples
    )
    return list(zip(X, y))

def run_single_simulation(seed: int):
    """Runs one full simulation and returns the error metrics."""
    np.random.seed(seed)
    N_INITIAL_TRAIN, N_FEATURES, N_DELETE, ALPHA = 4500, 5, 500, 0.1
    data = generate_synthetic_data(
        n_samples = N_INITIAL_TRAIN, 
        n_features = N_FEATURES
    )
    initial_data, data_to_delete = data[:-N_DELETE], data[-N_DELETE:]
    
    loss = lambda w, z: 0.5 * (z[0] @ w - z[1])**2 + 0.5 * ALPHA * np.dot(w, w)
    grad = lambda w, z: (z[0] @ w - z[1]) * z[0] + ALPHA * w
    hess = lambda w, z: np.outer(z[0], z[0]) + ALPHA * np.identity(N_FEATURES)
    
    model = MemoryPair(
        d = N_FEATURES, 
        loss = loss, 
        grad = grad, 
        hess = hess, 
        lam = ALPHA
    )
    model.fit(initial_data)
    model.delete(data_to_delete)
    w_deleted_approx = model.w.copy()

    data_after_delete = initial_data[:-N_DELETE]
    model_retrained = MemoryPair(
        d = N_FEATURES, 
        loss = loss, 
        grad = grad, 
        hess = hess, 
        lam = ALPHA
    )
    model_retrained.fit(data_after_delete)
    w_deleted_retrained = model_retrained.w.copy()
    
    error = np.linalg.norm(w_deleted_approx - w_deleted_retrained)
    norm = np.linalg.norm(w_deleted_retrained)
    return (error / norm) * 100 if norm != 0 else 0

if __name__ == "__main__":
    N_SIMULATIONS = 500
    print(f"🚀 Starting {N_SIMULATIONS} simulations...")

    errors = [run_single_simulation(seed=i) for i in range(N_SIMULATIONS)]

    mean_error = np.mean(errors)
    ci = st.t.interval(
        0.95,
        len(errors)-1,
        loc=mean_error,
        scale=st.sem(errors)
    )

    print("\n--- ✅ Simulation Analysis ---")
    print(f"Ran {len(errors)} successful simulations.")
    print("\n📊 [RESULTS] Incremental Deletion vs. Retraining")
    print(f"The relative error is, on average, {mean_error:.2f}%")
    print(f"95% Confidence Interval: [{ci[0]:.2f}%, {ci[1]:.2f}%]")

    # --- Save results to /results/ directory ---
    results_dir = os.path.join(
        os.path.dirname(os.path.dirname(__file__)),
        'results'
    )
    os.makedirs(
        results_dir, 
        exist_ok = True
        )

    # Save errors as .npy
    np.save(
        os.path.join(
            results_dir, 
            'errors.npy'
        ), 
        np.array(errors)
    )

    # Save summary statistics as .txt
    summary_path = os.path.join(
        results_dir, 
        'summary.txt'
    )
    with open(summary_path, 'w') as f:
        f.write("--- Simulation Analysis ---\n")
        f.write(f"Ran {len(errors)} successful simulations.\n")
        f.write("[RESULTS] Incremental Deletion vs. Retraining\n")
        f.write(f"The relative error is, on average, {mean_error:.2f}%\n")
        f.write(f"95% Confidence Interval: [{ci[0]:.2f}%, {ci[1]:.2f}%]\n")

🚀 Starting 500 simulations...

--- ✅ Simulation Analysis ---
Ran 500 successful simulations.

📊 [RESULTS] Incremental Deletion vs. Retraining
The relative error is, on average, 15.17%
95% Confidence Interval: [14.40%, 15.94%]


NameError: name '__file__' is not defined